In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
%run mainTNR.py

relTemp= 0.995
Iteration:  0  of  2000 , Trunc. Error: 2.00822e-06,0.000283112
Iteration:  100  of  2000 , Trunc. Error: 2.00822e-06,6.58429e-05
Iteration:  200  of  2000 , Trunc. Error: 2.00822e-06,4.71489e-05
Iteration:  300  of  2000 , Trunc. Error: 2.00822e-06,4.7101e-05
RGstep: 0 , Truncation Errors: 2.00822e-06, 4.7101e-05, 4.33853e-05, 0.000102462
Iteration:  0  of  2000 , Trunc. Error: 1.20523e-06,0.00369753
Iteration:  100  of  2000 , Trunc. Error: 1.20523e-06,7.01867e-05
Iteration:  200  of  2000 , Trunc. Error: 1.20523e-06,5.53399e-05
Iteration:  300  of  2000 , Trunc. Error: 1.20523e-06,5.53143e-05
RGstep: 1 , Truncation Errors: 1.20523e-06, 5.53143e-05, 8.85435e-05, 0.000119954
Iteration:  0  of  2000 , Trunc. Error: 1.39265e-06,0.00435041
Iteration:  100  of  2000 , Trunc. Error: 1.39265e-06,8.14519e-05
Iteration:  200  of  2000 , Trunc. Error: 1.39265e-06,6.88852e-05
Iteration:  300  of  2000 , Trunc. Error: 1.39265e-06,6.88755e-05
RGstep: 2 , Truncation Errors: 1.39265e

In [3]:
%run mainScInvTNR.py

Iteration:  0  of  2000 , Trunc. Error: 6.08743e-10,4.06838e-06
Iteration:  100  of  2000 , Trunc. Error: 6.08743e-10,2.94729e-06
Iteration:  200  of  2000 , Trunc. Error: 6.08743e-10,2.48122e-06
Iteration:  300  of  2000 , Trunc. Error: 6.08743e-10,2.1954e-06
Iteration:  400  of  2000 , Trunc. Error: 6.08743e-10,1.98306e-06
Iteration:  500  of  2000 , Trunc. Error: 6.08743e-10,1.81134e-06
Iteration:  600  of  2000 , Trunc. Error: 6.08743e-10,1.67127e-06
Iteration:  700  of  2000 , Trunc. Error: 6.08743e-10,1.55939e-06
Iteration:  800  of  2000 , Trunc. Error: 6.08743e-10,1.4727e-06
Iteration:  900  of  2000 , Trunc. Error: 6.08743e-10,1.40773e-06
Iteration:  1000  of  2000 , Trunc. Error: 6.08743e-10,1.36077e-06
Iteration:  1100  of  2000 , Trunc. Error: 6.08743e-10,1.32807e-06
Iteration:  1200  of  2000 , Trunc. Error: 6.08743e-10,1.30612e-06
Iteration:  1300  of  2000 , Trunc. Error: 6.08743e-10,1.29188e-06
Iteration:  1400  of  2000 , Trunc. Error: 6.08743e-10,1.2829e-06
RGstep: 0 

In [4]:
scDims

array([-0.        ,  0.12704589,  1.00024473,  1.12222162,  1.12333985,
        1.99851092,  1.99872306,  1.99913345,  1.99941232,  2.11939745,
        2.11999277,  2.12728698,  2.99244553,  2.99264387,  2.99320731,
        2.9946286 ,  2.99944181,  3.10623731,  3.11430485,  3.12301525,
        3.12305146,  3.12481681,  3.12538312,  3.97665396,  3.97973526,
        3.98069923,  3.98091394,  3.99101189,  3.99132524,  3.99594198,
        3.99598677,  4.00102818,  4.0656465 ,  4.11009329,  4.11150933,
        4.11646297,  4.1191813 ,  4.12122506,  4.12195357,  4.12279122,
        4.12411147,  4.94884227,  4.95691295,  4.95985151,  4.96244066,
        4.97608263,  4.9801529 ,  4.98548764,  4.9876223 ,  4.99344789,
        4.99367804,  4.99522515,  4.99835161,  4.99842436,  5.00692568,
        5.06685636,  5.07381003,  5.10829708,  5.10874253,  5.11110572,
        5.12072758,  5.12072758,  5.12094167,  5.1212284 ,  5.12204152,
        5.12387944,  5.12932713,  5.12936624,  5.90765802,  5.92

In [1]:
# #Todo: diagonalize in python

# import h5py
# f = h5py.File('rT.h5', 'r')
# a_group_key = list(f.keys())[0]
# rT=np.array(f[a_group_key])

# f = h5py.File('gT.h5', 'r')
# a_group_key = list(f.keys())[0]
# gT=np.array(f[a_group_key])